# Scrittopoli - Giornata 1

In [1]:
%matplotlib inline

# Import the necessary packages and modules
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.dates  as mpdt
import matplotlib.ticker as mptk

import numpy as np
import pytz
mytz=pytz.timezone("Europe/Rome")
import pandas as pd
import datetime as dt
import math
import importlib
import random
import itertools
import csv
import mylib.spreadsheet as ms
import mylib.elo as melo


In [2]:
day="20171209_192536"
calendario_csv="data/calendario_%s.csv" % day
gironi_csv="data/gironi_%s.csv" % day
tabellone_csv="data/tabellone_%s.csv" % day
ranking_csv="data/ranking_%s.csv" % day
scrittopoli_csv="data/scrittopoli_%s.csv" % day
elo_giocatori_csv="data/elo_giocatori_%s.csv" % day
elo_squadre_csv="data/elo_squadre_%s.csv" % day

calendario=pd.read_csv(calendario_csv,sep=":",quotechar='"',header=0,index_col=1).sort_index()
calendario=calendario.set_index(["girone","partita"], append=True)

gironi=pd.read_csv(gironi_csv,sep=":",quotechar='"',header=0)

tabellone=pd.read_csv(tabellone_csv,sep=",",quotechar='"',header=0,index_col=0)
elo_giocatori=pd.read_csv(elo_giocatori_csv,sep=",",quotechar='"',header=0,index_col=0)

ranking=pd.read_csv(ranking_csv,sep=",",quotechar='"',header=0)
ranking=ranking[ranking["squadra"]!="(riposo)"]
ranking=ranking.set_index("squadra")

scrittopoli=pd.read_csv(scrittopoli_csv,sep=",",quotechar='"',header=0)
scrittopoli=scrittopoli.set_index(["squadra","giocatore"])

elo_squadre=pd.read_csv(elo_squadre_csv,sep=",",quotechar='"',header=0)
elo_squadre=elo_squadre.set_index(["squadra","giocatore"])



## Come si svolgono gli incontri

Ogni capitano presenta la formazione (due titolari e una riserva). I due titolari vengono accoppiati agli avversari sulla base del punteggio Elo (i due più alti tra loro e i due più bassi tra loro).

Inseriscono i due racconti. Cose da controllare:
* link al racconto;
* tag;
* lunghezza del racconto;
* link al commento;
* commento valido.

Per la votazione ogni squadra presenta l'elenco dei voti da dare a quelle degli altri gironi. Per ogni partita danno quattro punteggi. Oltre a queste ci sono le penalità dell'arbitro.


In [3]:
calendario[calendario["squadra 2"]!="(riposo)"].sort_index()

squadra 1                      squadra 2
giornata girone partita                                                   
1        A      1               Green House                     I Bananari
         B      1             Living Stones                   La threesome
         C      1               D Euforiche             Retroguardia Trash
                2            the 3 BrOthers             Armata Brancaleone
2        A      1               Green House  Gli scrittori del quartierino
         B      1             Living Stones                    Le TreMende
         C      1               D Euforiche                 the 3 BrOthers
                2        Retroguardia Trash             Armata Brancaleone
3        A      2                I Bananari  Gli scrittori del quartierino
         B      2              La threesome                    Le TreMende
         C      1               D Euforiche             Armata Brancaleone
                2        Retroguardia Trash                 the 3 BrOthers

## Fase 1. Dalle formazioni ai match

In [7]:
squadre=scrittopoli.copy()
squadre=squadre.reset_index()[["squadra","giocatore"]].set_index("squadra")

# Versione random di prova, da sostituire con lettura della formazione via csv o spreadsheet
data=[]
for sq in squadre.index.unique():
    capitano=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
    riserva=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
    titolari=list(squadre[squadre["giocatore"]!=riserva].loc[sq]["giocatore"])
    data.append([sq,capitano,riserva]+titolari)
formazioni=pd.DataFrame(data,columns=["squadra","capitano","riserva","titolare 1","titolare 2"])


In [8]:
formazioni=formazioni.set_index("squadra")
formazioni

,capitano,riserva,titolare 1,titolare 2
squadra,,,,
Green House,Bango Skank,Plata,Bango Skank,Joyopi
La threesome,mina99,mina99,Komorebi,Luna
I Bananari,Niko,simone volponi,Niko,massimopud
Armata Brancaleone,Skorpiòs,AdStr,Ella F.,Skorpiòs
Living Stones,Macleo,Macleo,Vincenzo Iennaco,Ljuset
Retroguardia Trash,Eudes,Eudes,AlexComan,Andrea28
Le TreMende,sefora,sefora,Emy,Thea
Gli scrittori del quartierino,Federico72,camparino,Federico72,Marty12
the 3 BrOthers,AryaSophia,AryaSophia,ettore70,ITG


In [75]:
def parametri_giornata(giornata,calendario,elo):
    rank=elo.copy().reset_index().set_index("giocatore")["rank %d" % (giornata-1)]
    cal=calendario[calendario["squadra 2"]!="(riposo)"].loc[1]
    return cal,rank

def sort_by_rank(rank,a,b):
    if rank[a]>rank[b]: return [a,b]
    if rank[a]<rank[b]: return [b,a]
    L=[a,b]
    random.shuffle(L)
    return L

def calcola_accoppiamenti(giornata,formazioni,calendario,elo_squadre):
    cal,rank=parametri_giornata(1,calendario,elo_squadre)
    data=[]
    for girone,partita in cal.index:
        for n in [1,2]:
            squadra=cal.loc[(girone,partita)]["squadra %d" % n]
            riserva=formazioni.loc[squadra]["riserva"]
            capitano=formazioni.loc[squadra]["capitano"]
            titolare_1=formazioni.loc[squadra]["titolare 1"]
            titolare_2=formazioni.loc[squadra]["titolare 2"]
            data.append([girone,partita,squadra,capitano,riserva]+sort_by_rank(rank,titolare_1,
                                                                               titolare_2))    
    X=pd.DataFrame(data,
                   columns=["girone","partita","squadra","capitano","riserva",
                            "match 1","match 2"])
    X=X.set_index(["girone","partita"]).sort_index()
    return X

accoppiamenti=calcola_accoppiamenti(1,formazioni,calendario,elo_squadre)
accoppiamenti


squadra     capitano         riserva  \
girone partita                                                    
A      1               Green House  Bango Skank           Plata   
       1                I Bananari         Niko  simone volponi   
B      1             Living Stones       Macleo          Macleo   
       1              La threesome       mina99          mina99   
C      1               D Euforiche         Kuno          queffe   
       1        Retroguardia Trash        Eudes           Eudes   
       2            the 3 BrOthers   AryaSophia      AryaSophia   
       2        Armata Brancaleone     Skorpiòs           AdStr   

                         match 1    match 2  
girone partita                               
A      1             Bango Skank     Joyopi  
       1              massimopud       Niko  
B      1        Vincenzo Iennaco     Ljuset  
       1                Komorebi       Luna  
C      1                Marcello       Kuno  
       1                Andrea28  AlexComan  
       2                     ITG   ettore70  
       2                 Ella F.   Skorpiòs

## Fase 2. Creazione del tabellone per voti e controlli

In [235]:
importlib.reload(ms)
s=ms.N2017Spreadsheet()

old_data=s.get_giornata(1)
s.set_giornata(1,accoppiamenti,gironi,old_data)


## Fase 3. Recupero dati

In [240]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
q=s.get_giornata(1)


base        verifiche                         \
                 ingresso riserva titolo lunghezza tag link link commento   
titolare                                                                    
Bango Skank                                   3000                          
Joyopi                                                                      
massimopud                                                                  
Niko                                                                        
Vincenzo Iennaco                                                            
Ljuset                                                                      
Komorebi                                                                    
Luna                                                                        
Marcello                                                                    
Kuno                                                                        
Andrea28                                                                    
AlexComan                                                                   
ITG                                                                         
ettore70                                                                    
Ella F.                                                                     
Skorpiòs                                                                    

                                           A             \
                 commento valido Green House I Bananari   
titolare                                                  
Bango Skank                                               
Joyopi                                                    
massimopud                                                
Niko                                                      
Vincenzo Iennaco                                          
Ljuset                                                    
Komorebi                                                  
Luna                                                      
Marcello                                                  
Kuno                                                      
Andrea28                                                  
AlexComan                                                 
ITG                                                       
ettore70                                                  
Ella F.                                                   
Skorpiòs                                                  

                                                           B               \
                 Gli scrittori del quartierino Living Stones La threesome   
titolare                                                                    
Bango Skank                                                                 
Joyopi                                                                      
massimopud                                                                  
Niko                                                                        
Vincenzo Iennaco                                                            
Ljuset                                                                      
Komorebi                                                                    
Luna                                                                        
Marcello                                                                    
Kuno                                                                        
Andrea28                                                                    
AlexComan                                                                   
ITG                                                                         
ettore70                                                                    
Ella F.                                                                     
Skorpiòs                                                 

In [255]:
### Simulazione di riempimento del tabellone
import string


def random_title(x):
    size=random.randint(10,20)
    chars=string.ascii_lowercase +string.ascii_uppercase + string.digits
    chars+="               "
    S=''.join(random.choice(chars) for _ in range(size))
    S=S.strip()
    S=" ".join(S.split())
    return S

def random_len(x):
    return random.randint(300,8200)

def random_riserva(x):
    if random.choice([True,False]):
        return ""
    return "12-08 23:23"

def random_verifica(x):
    if random.choice([True,False,True,True,True,True,True,True,True,True,True]):
        return "x"
    return ""

q["base","titolo"]=q["base","titolo"].apply(random_title)
q["base","ingresso riserva"]=q["base","ingresso riserva"].apply(random_riserva)
q["verifiche","lunghezza"]=q["verifiche","lunghezza"].apply(random_len)
q["verifiche","link"]=q["verifiche","link"].apply(random_)
q

#accoppiamenti

base                      verifiche           \
                 ingresso riserva               titolo lunghezza tag link   
titolare                                                                    
Bango Skank                                 4RV OQ9nWt      6281            
Joyopi                              PX RQ 0Dosw CY BEe      6155            
massimopud                                  KTiQN Bh v      6357            
Niko                  12-08 23:23      14p5EVupcK01zYb      6651            
Vincenzo Iennaco      12-08 23:23         pW4 gMBPYAJf      1240            
Ljuset                12-08 23:23            h i spLxo       426            
Komorebi              12-08 23:23          a8 0W jUl U      3513            
Luna                  12-08 23:23       ZklQ8 m9O jXqs      1116            
Marcello              12-08 23:23      Y7VN3J uVLFOeJb      1954            
Kuno                  12-08 23:23     3DGq 7T8 SUY O8t       315            
Andrea28              12-08 23:23         Nqw LVHnh H0      2137            
AlexComan             12-08 23:23           oeEf Rhzc4       368            
ITG                   12-08 23:23          S rB R8JGku      1084            
ettore70                                  NN tGTWXy4 p      4519            
Ella F.                            W oTgcmT7rMigaBC9vP      4989            
Skorpiòs                             un6d EQ5YSIW1dqft       919            

                                                         A             \
                 link commento commento valido Green House I Bananari   
titolare                                                                
Bango Skank                                                             
Joyopi                                                                  
massimopud                                                              
Niko                                                                    
Vincenzo Iennaco                                                        
Ljuset                                                                  
Komorebi                                                                
Luna                                                                    
Marcello                                                                
Kuno                                                                    
Andrea28                                                                
AlexComan                                                               
ITG                                                                     
ettore70                                                                
Ella F.                                                                 
Skorpiòs                                                                

                                                           B               \
                 Gli scrittori del quartierino Living Stones La threesome   
titolare                                                                    
Bango Skank                                                                 
Joyopi                                                                      
massimopud                                                                  
Niko                                                                        
Vincenzo Iennaco                                                            
Ljuset                                                                      
Komorebi                                                                    
Luna                                                                        
Marcello                                                                    
Kuno                                                                        
Andrea28                                                                    
AlexComan                                                                   
ITG                   

In [139]:
A=[(1,2),(3,4)]

collections.OrderedDict(A)

OrderedDict([(1, 2), (3, 4)])

In [ ]:
## classifica e ranking